In [2]:
import numpy as np
from matplotlib import pyplot as plt
import json
from tqdm import tqdm
import sys
import corner
import matplotlib.image as mpimg

sys.path.append("../")

import tdpy

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
def gen_arr(config, bin=True, n=12000, secondary=True):
    n_secondary = config["n_secondary"]
    l = bin*len(config["binary"].keys()) + secondary*n_secondary*len(config["secondary_0"].keys()) + 1
    arr = np.zeros((n, l))*np.nan
    keys = []
    keys.append("m_star")
    if secondary:
        for i in range(0, n_secondary):
            for key in config["secondary_0"].keys():
                keys.append(f"{i}/{key}")
    if bin:
        for key in config["binary"].keys():
            keys.append(f"bin/{key}")
    
    return arr, keys




def add_to_array(arr, config, keys, i, bin=True, n=12000, secondary=True):
    if not np.any(arr):
        arr, keys = gen_arr(config, bin=bin, n=n, secondary=secondary)
        
    n_secondary = config["n_secondary"]
    arr[i, 0] = config["m_star"]
    j = 1
    
    if secondary:
        for k in range(0, n_secondary):
            for key in config["secondary_0"].keys():
                arr[i, j] = config[f"secondary_{k}"][key]
                j += 1
    if bin:
        for key in config["binary"].keys():
            arr[i, j] = config["binary"][key]
            j += 1
    
    return arr, keys
    

In [6]:
dir = "bin_inj"
system = "KOI-351"

keys = None
arr = None
j = 0
stable = np.zeros(1000)

for i in tqdm(range(0, 1000)):
    try:
        with open(f"../output/{dir}/{system}/{i}/config.json") as f:
            cfg = json.load(f)
            arr, keys = add_to_array(arr, cfg, keys, j, bin=True, secondary=False)
            
        elements = np.load(f"../output/{dir}/{system}/{i}/elements.npy")
        a_i = elements[0, :, 0]
        a_f = elements[-1, :, 0]
        stable[i] = np.all(((a_i-a_f)/a_i) < 0.1)
        j += 1

    except Exception as e:
#         print(e)
        pass
arr = arr[:j, :]
stable = stable[:j]
stable = np.logical_not(np.logical_not(stable))
keys = np.array(keys)
arr += np.random.random(arr.shape) * 1e-20

m_tot = arr[:, keys=="bin/m1"] + arr[:, keys=="bin/m2"]
q = arr[:, keys=="bin/m1"] / m_tot
arr[:, keys=="bin/m1"] = m_tot
arr[:, keys=="bin/m2"] = q
keys[keys=="bin/m1"] = "bin/m_tot"
keys[keys=="bin/m2"] = "bin/q"


100%|██████████| 1000/1000 [00:02<00:00, 340.36it/s]


In [5]:
idxs = np.array(["inc" not in key for key in keys])
# idxs = np.array([("inc" not in key) and ("/e" not in key) for key in keys])
# idxs = np.array([True for key in keys])


param_labels = np.char.replace(np.array(keys)[idxs], "_", "-")



# labels of the populations
listlablpopl = ['stable', 'not stable']

# number of populations
numbpopl = len(listlablpopl)


# number of features
numbfeat = np.sum(idxs)

# generate samples


# labels of the parameters
listlablpara = []
for k in range(numbfeat):
    listlablpara.append([param_labels[k], ''])

typepgrp = 'both'
    
listcolrpopl = ["red", "gray"]
    
tdpy.plot_grid( [arr[stable, :][: ,idxs], arr[np.logical_not(stable),:][:,idxs]], listlablpara, listlablpopl=listlablpopl, typeplottdim='scat', pathbase=f"../figs/corner_{system}_", listcolrpopl =listcolrpopl )
# plt.savefig("../figs/corner.pdf")


Number of samples in the populations:
not stable
0
stable
0
Writing to ../figs/corner_KOI-351_pmar_scat_None.png...


In [ ]:
plt.imshow(mpimg.imread('../figs/corner_KOI-351_pmar_scat_None.png'))

In [52]:
arr[stable, :][:, idxs].shape

(14, 31)

In [22]:
idxs = np.array([("inc" not in key) and ("/e" not in key) for key in keys])



param_labels = np.char.replace(np.array(keys)[idxs], "_", "-")



# labels of the populations
listlablpopl = ['stable', 'not stable']

# number of populations
numbpopl = len(listlablpopl)


# number of features
numbfeat = np.sum(idxs)

# generate samples


# labels of the parameters
listlablpara = []
for k in range(numbfeat):
    listlablpara.append([param_labels[k], ''])

typepgrp = 'both'
    
j 
tdpy.plot_grid( [arr[stable, :][: ,idxs][:, :j], arr[np.logical_not(stable),:][:,idxs][:, :j]], listlablpara[:j], listlablpopl=listlablpopl, typeplottdim='scat', pathbase="../figs/corner")
# plt.savefig("../figs/corner.pdf")
# plt.show()

Number of samples in the populations:
not stable
169
stable
831
Writing to ../figs/cornerpmar_scat_None.png...


array([[6.36531282e-21, 6.00896477e-21, 1.54466773e-21, ...,
        7.69877385e-21, 5.01658692e-21, 4.02435015e-21],
       [7.99278057e-21, 7.06395162e-21, 5.58262707e-21, ...,
        2.80606189e-21, 1.16639642e-21, 3.86933171e-21],
       [6.33603307e-21, 8.67720397e-21, 1.05688466e-21, ...,
        3.55359739e-21, 6.75482858e-21, 9.79935197e-21],
       ...,
       [6.18044631e-21, 4.51022624e-21, 4.78983885e-21, ...,
        2.05398366e-21, 1.04960145e-21, 5.50505039e-21],
       [5.72203910e-22, 9.49231428e-21, 4.98316740e-21, ...,
        7.07835013e-21, 8.51138939e-21, 2.32587116e-21],
       [9.34785075e-21, 2.75058350e-21, 9.13041580e-21, ...,
        7.88077819e-21, 7.08283891e-21, 8.45677493e-21]])